# Purpose

The purpose of this notebook is to illustrate how get data from `data.app_data.DataAlpacaPocCat`.

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import datetime
from functools import reduce
#from datetime import timedelta
import pymongo
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from supervised.automl import AutoML
import json

sys.path.append("../src/")
from tradingdmp.data.clf.price_perc_chg.app_data import DataAlpacaPocCat
from tradingdmp.data.utils.prep_data import PrepData
from tradingdmp.data.utils.raw_data import RawData
from tradingdmp.model.clf.app_model import MljarAutoMl

In [2]:
pd.set_option('display.max_columns', None)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Import config

In [3]:
# Load mongodbkey from config
with open("../.config/config.json") as f:
    data = json.load(f)
    mongodbkey = data["mongodbkey"]


In [4]:
# Load model config data
with open("config_model.json") as f:
    data = json.load(f)

    seed = data["seed"]
    bins = data["bins"]
    bin_labels = data["bin_labels"]
    test_size = data["test_size"]

    dt_start = data["dt_start"]
    dt_end = data["dt_end"]
    ticker_list = data["ticker_list"]
    n_ppc_per_row = data["n_ppc_per_row"]

    mode = data["mode"]
    eval_metric = data["eval_metric"]
    total_time_limit = data["total_time_limit"]
    algorithms = data["algorithms"]



In [8]:
# Get data from mongodb
print("Get data from mongodb...")
adata = DataAlpacaPocCat(mongodbkey)
df_x, df_y = adata.get_data_cached(
    ticker_list,
    datetime.datetime.strptime(dt_start, "%Y-%m-%d"),
    datetime.datetime.strptime(dt_end, "%Y-%m-%d"),
    dt_end_required=False,
    return_last_date=False,
    return_nonlast_dates=True,
    return_training_dfs=True,
    return_date_col=True,
    return_ticker_col=False,
    bins=bins,
    bin_labels=bin_labels,
    n_ppc_per_row=n_ppc_per_row,
)

Get data from mongodb...


# With return_last_date, return_training_dfs

In [5]:
# The end date should be a weekday (t-1 if we trade on t)
dt_end = datetime.datetime.strptime("2021-07-15", '%Y-%m-%d')
dt_start = dt_end - datetime.timedelta(days = 100)
dt_end
dt_start

datetime.datetime(2021, 7, 15, 0, 0)

datetime.datetime(2021, 4, 6, 0, 0)

In [6]:
adata = DataAlpacaPocCat(mongodbkey)

### Trading: Get only last dates for data, where the last day is given

In [7]:
df_x, df_y = adata.get_data(
    ticker_list,
    dt_start,
    dt_end,
    dt_end_required=True,
    return_last_date=True,
    return_nonlast_dates=False,
    return_training_dfs=True,
    return_date_col=True,
    return_ticker_col=True,
    n_ppc_per_row=10
)

Querying ticker: TIF failed
Querying ticker: GV failed
Querying ticker: HMSY failed
Querying ticker: QEP failed
Querying ticker: EIGI failed
Querying ticker: CXO failed


In [8]:
df_x

,ticker,date,av_open,av_close,av_high,av_low,av_volume,av_adjusted_close,av_dividend_amount,av_split_coefficient,fv_Sector,fv_Industry,fv_PE,fv_EPSttm,fv_InsiderOwn,fv_ShsOutstand,fv_PerfWeek,fv_MarketCap,fv_ForwardPE,fv_EPSnextY,fv_InsiderTrans,fv_ShsFloat,fv_PerfMonth,fv_Income,fv_PEG,fv_EPSnextQ,fv_InstOwn,fv_ShortFloat,fv_PerfQuarter,fv_Sales,fv_PS,fv_EPSthisY,fv_ShortRatio,fv_PerfHalfY,fv_Booksh,fv_PB,fv_ROA,fv_TargetPrice,fv_PerfYear,fv_Cashsh,fv_PC,fv_EPSnext5Y,fv_ROE,fv_PerfYTD,fv_Dividend,fv_PFCF,fv_EPSpast5Y,fv_ROI,fv_52WHigh,fv_Beta,fv_QuickRatio,fv_Salespast5Y,fv_GrossMargin,fv_52WLow,fv_ATR,fv_Employees,fv_CurrentRatio,fv_SalesQQ,fv_OperMargin,fv_RSI14,fv_Optionable,fv_DebtEq,fv_EPSQQ,fv_ProfitMargin,fv_RelVolume,fv_PrevClose,fv_Shortable,fv_LTDebtEq,fv_Payout,fv_AvgVolume,fv_Price,fv_Recom,fv_SMA20,fv_SMA50,fv_SMA200,fv_Volume,fv_Change,av_open_0,av_close_0,av_high_0,av_low_0,av_volume_0,av_adjusted_close_0,av_dividend_amount_0,av_split_coefficient_0,av_open_1,av_close_1,av_high_1,av_low_1,av_volume_1,av_adjusted_close_1,av_dividend_amount_1,av_split_coefficient_1,av_open_2,av_close_2,av_high_2,av_low_2,av_volume_2,av_adjusted_close_2,av_dividend_amount_2,av_split_coefficient_2,av_open_3,av_close_3,av_high_3,av_low_3,av_volume_3,av_adjusted_close_3,av_dividend_amount_3,av_split_coefficient_3,av_open_4,av_close_4,av_high_4,av_low_4,av_volume_4,av_adjusted_close_4,av_dividend_amount_4,av_split_coefficient_4,av_open_5,av_close_5,av_high_5,av_low_5,av_volume_5,av_adjusted_close_5,av_dividend_amount_5,av_split_coefficient_5,av_open_6,av_close_6,av_high_6,av_low_6,av_volume_6,av_adjusted_close_6,av_dividend_amount_6,av_split_coefficient_6,av_open_7,av_close_7,av_high_7,av_low_7,av_volume_7,av_adjusted_close_7,av_dividend_amount_7,av_split_coefficient_7,av_open_8,av_close_8,av_high_8,av_low_8,av_volume_8,av_adjusted_close_8,av_dividend_amount_8,av_split_coefficient_8,av_open_9,av_close_9,av_high_9,av_low_9,av_volume_9,av_adjusted_close_9,av_dividend_amount_9,av_split_coefficient_9
0,AAP,2021-07-15,208.40,209.25,210.5400,207.57,547188.0,209.25,0.0,1.0,Consumer Cyclical,Specialty Retail,22.43,9.33,0.40,65690000.0,0.84,1.331000e+10,17.12,11.72,-1.09,65160000.0,4.28,6.354000e+08,1.81,2.93,0.0,5.13,10.45,1.074000e+10,1.24,4.4,4.10,21.83,53.31,3.93,5.3,217.75,45.64,13.84,15.12,12.41,17.5,32.85,1.91,13.79,2.2,11.8,-2.19,1.38,0.4,0.7,44.7,49.23,3.61,40000.0,1.3,23.4,8.2,58.62,Yes,0.00,347.8,5.9,0.67,210.16,Yes,0.30,10.7,815590.0,209.25,2.2,2.21,4.40,20.23,547188.0,-0.43,-0.001988,0.018524,0.007611,0.003331,0.076242,0.018524,0.0,0.0,0.015160,0.010099,0.009466,0.016550,-0.145680,0.010099,0.0,0.0,0.007036,-0.011135,-0.002889,-0.008405,0.035195,-0.011135,0.0,0.0,-0.014116,0.003546,-0.003705,0.000630,0.275241,0.003546,0.0,0.0,-0.002700,-0.009215,-0.000334,-0.005518,-0.042695,-0.009215,0.0,0.0,0.029874,0.028722,0.018979,0.022097,-0.009210,0.028722,0.0,0.0,-0.002863,-0.007214,0.001217,0.005524,-0.422216,-0.007214,0.0,0.0,-0.006590,-0.013967,-0.008460,-0.011082,0.822481,-0.013967,0.0,0.0,-0.004691,0.005695,-0.006128,-0.001532,-0.100519,0.005695,0.0,0.0,-0.007855,-0.004330,-0.001376,-0.004460,-0.116544,-0.004330,0.0,0.0
1,ABC,2021-07-15,112.79,113.16,114.0300,112.79,757075.0,113.16,0.0,1.0,Healthcare,Medical Distribution,0.00,-18.43,0.20,204920000.0,-1.12,2.311000e+10,10.70,17.02,-42.37,146120000.0,-4.93,-3.746500e+09,0.00,2.02,67.1,1.88,-4.94,1.962800e+11,0.12,-530.9,2.67,7.14,-1.37,0.00,-8.4,131.85,11.05,32.53,3.48,12.30,-806.0,15.75,1.56,24.89,8.7,-105.2,-10.09,0.53,0.7,6.9,2.8,23.00,1.89,22000.0,1.1,3.7,-2.3,38.97,Yes,0.00,-54.7,-1.9,0.73,113.56,Yes,0.00,0.0,1030000.0,113.16,2.2,-2.11,-3.47,3.95,750155.0,-0.35,0.002443,0.017032,0.016768,0.011266,0.513980,0.017032,0.0,0.0,0.011140,0.001546,-0.000940,0.001305,-0.344924,0.001546,0.0,0.0,-0.001205,-0.010290,-0.006757,-0.020774,0.233016,-0.010290,0.0,0.0,-0.009393,-0.003292,-0.000086,0.017397,-0.153024,-0.003292,0.0,0.0,-0.006873,-0.005216,-0.011023,-0.011865,-0.210765,-0.005216,0.0,0.0,

In [9]:
df_y

,y
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
374,NaN
375,NaN
376,NaN
377,NaN


### Training: Get only non-last dates for any data

In [10]:
df_x, df_y = adata.get_data(
    ticker_list,
    dt_start,
    dt_end,
    dt_end_required=False,
    return_last_date=False,
    return_nonlast_dates=True,
    return_training_dfs=True,
    return_date_col=True,
    return_ticker_col=True,
    n_ppc_per_row=10
)

Querying ticker: TIF failed
Querying ticker: GV failed
Querying ticker: HMSY failed
Querying ticker: QEP failed
Querying ticker: EIGI failed
Querying ticker: CXO failed


In [11]:
df_x

,ticker,date,av_open,av_close,av_high,av_low,av_volume,av_adjusted_close,av_dividend_amount,av_split_coefficient,fv_Sector,fv_Industry,fv_PE,fv_EPSttm,fv_InsiderOwn,fv_ShsOutstand,fv_PerfWeek,fv_MarketCap,fv_ForwardPE,fv_EPSnextY,fv_InsiderTrans,fv_ShsFloat,fv_PerfMonth,fv_Income,fv_PEG,fv_EPSnextQ,fv_InstOwn,fv_ShortFloat,fv_PerfQuarter,fv_Sales,fv_PS,fv_EPSthisY,fv_ShortRatio,fv_PerfHalfY,fv_Booksh,fv_PB,fv_ROA,fv_TargetPrice,fv_PerfYear,fv_Cashsh,fv_PC,fv_EPSnext5Y,fv_ROE,fv_PerfYTD,fv_Dividend,fv_PFCF,fv_EPSpast5Y,fv_ROI,fv_52WHigh,fv_Beta,fv_QuickRatio,fv_Salespast5Y,fv_GrossMargin,fv_52WLow,fv_ATR,fv_Employees,fv_CurrentRatio,fv_SalesQQ,fv_OperMargin,fv_RSI14,fv_Optionable,fv_DebtEq,fv_EPSQQ,fv_ProfitMargin,fv_RelVolume,fv_PrevClose,fv_Shortable,fv_LTDebtEq,fv_Payout,fv_AvgVolume,fv_Price,fv_Recom,fv_SMA20,fv_SMA50,fv_SMA200,fv_Volume,fv_Change,av_open_0,av_close_0,av_high_0,av_low_0,av_volume_0,av_adjusted_close_0,av_dividend_amount_0,av_split_coefficient_0,av_open_1,av_close_1,av_high_1,av_low_1,av_volume_1,av_adjusted_close_1,av_dividend_amount_1,av_split_coefficient_1,av_open_2,av_close_2,av_high_2,av_low_2,av_volume_2,av_adjusted_close_2,av_dividend_amount_2,av_split_coefficient_2,av_open_3,av_close_3,av_high_3,av_low_3,av_volume_3,av_adjusted_close_3,av_dividend_amount_3,av_split_coefficient_3,av_open_4,av_close_4,av_high_4,av_low_4,av_volume_4,av_adjusted_close_4,av_dividend_amount_4,av_split_coefficient_4,av_open_5,av_close_5,av_high_5,av_low_5,av_volume_5,av_adjusted_close_5,av_dividend_amount_5,av_split_coefficient_5,av_open_6,av_close_6,av_high_6,av_low_6,av_volume_6,av_adjusted_close_6,av_dividend_amount_6,av_split_coefficient_6,av_open_7,av_close_7,av_high_7,av_low_7,av_volume_7,av_adjusted_close_7,av_dividend_amount_7,av_split_coefficient_7,av_open_8,av_close_8,av_high_8,av_low_8,av_volume_8,av_adjusted_close_8,av_dividend_amount_8,av_split_coefficient_8,av_open_9,av_close_9,av_high_9,av_low_9,av_volume_9,av_adjusted_close_9,av_dividend_amount_9,av_split_coefficient_9
0,AAP,2021-04-20,198.00,193.91,198.5200,190.9100,1768824.0,193.91,0.0,1.0,Consumer Cyclical,Specialty Retail,27.14,7.14,0.5,67700000.0,3.47,1.246000e+10,17.33,13.90,0.00,64870000.0,4.25,493000000.0,1.94,2.59,0.0,4.63,16.86,1.011000e+10,1.23,4.4,2.97,25.11,52.58,3.69,4.1,185.19,68.03,12.99,14.92,14.01,13.5,23.11,0.52,19.29,2.2,11.8,-1.15,1.35,0.4,0.7,44.4,72.50,4.40,40000.0,1.3,12.0,6.9,68.00,Yes,0.0,19.7,4.9,1.75,192.54,Yes,0.29,14.8,1010000.0,193.91,2.2,4.54,10.81,21.82,1768824.0,0.71,0.017820,-0.005608,0.001409,0.002048,-0.505730,-0.005608,0.0,0.0,-0.007805,-0.011609,-0.009094,-0.008009,0.230661,-0.011609,0.0,0.0,-0.004862,0.019224,0.007703,0.009131,0.517760,0.019224,0.0,0.0,0.010375,0.022232,0.022064,0.015504,-0.264720,0.022232,0.0,0.0,0.025645,-0.003456,0.002016,0.007552,0.037326,-0.003456,0.0,0.0,-0.006887,0.001921,0.003070,0.008952,0.211333,0.001921,0.0,0.0,0.004374,0.009000,0.003747,0.005291,-0.208453,0.009000,0.0,0.0,0.015880,0.018262,0.015615,0.013079,0.117505,0.018262,0.0,0.0,0.012077,-0.001970,0.015530,0.009446,0.280567,-0.001970,0.0,0.0,0.022780,0.007115,0.011979,-0.007486,1.038847,0.007115,0.0,0.0
1,AAP,2021-04-21,194.45,199.87,202.5400,194.4500,1220753.0,199.87,0.0,1.0,Consumer Cyclical,Specialty Retail,27.97,7.14,0.5,67700000.0,6.44,1.284000e+10,17.86,13.90,0.00,64870000.0,9.22,493000000.0,2.00,2.59,0.0,4.63,21.00,1.011000e+10,1.27,4.4,2.91,30.42,52.58,3.80,4.1,185.19,76.08,12.99,15.38,14.01,13.5,26.89,0.50,19.89,2.2,11.8,0.68,1.35,0.4,0.7,44.4,77.80,4.72,40000.0,1.3,12.0,6.9,73.67,Yes,0.0,19.7,4.9,1.19,193.91,Yes,0.29,14.8,1030000.0,199.87,2.2,7.43,13.74,25.36,1220753.0,3.07,-0.007805,-0.011609,-0.009094,-0.008009,0.230661,-0.011609,0.0,0.0,-0.004862,0.019224,0.007703,0.009131,0.517760,0.019224,0.0,0.0,0.010375,0.022232,0.022064,0.015504,-0.264720,0.022232,0.0,0.0,0.025645,-0.003456,0.002016,0.007552,0.037326,-0.003456,0.0,0.0,-0.006887,0.001921,0.003070,0.008952,0.211333,0.001921,0.0,0.0,0.004374,0.009000,0.003747,0.005291,-0.

In [12]:
df_y

,y
0,4_lg_inc
1,2_no_chg
2,2_no_chg
3,1_sm_dec
4,2_no_chg
...,...
22447,2_no_chg
22448,1_sm_dec
22449,2_no_chg
22450,0_lg_dec
